In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - NLLB (No Language Left Behind)

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_nllb.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_nllb.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_nllb.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
Open in Vertex AI Workbench
    </a>
    (a Python-3 CPU notebook is recommended)
  </td>
</table>

## Overview

This notebook demonstrates deploying the pre-trained [NLLB](https://huggingface.co/facebook/nllb-200-3.3B) model on Vertex AI for online prediction.

### Objective

- Upload the model to [Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction).
- Deploy the model on [Endpoint](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints).
- Run online predictions for translation.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Setup environment

**NOTE**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

### Colab only

In [ ]:
if "google.colab" in str(get_ipython()):
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Setup Google Cloud project

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).

1. [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs.

Fill following variables for experiments environment:

In [ ]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# Google Cloud Storage bucket URI.
GCS_BUCKET = "gs://"  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = "us-central1"  # @param {type:"string"}

Initialize Vertex AI API:

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

### Define constants

In [ ]:
# The pre-built serving docker image.
# The model artifacts are embedded within the container, except for model weights which will be downloaded during deployment.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-transformers-serve:20231022_0832_RC00"

### Define common functions

In [ ]:
def deploy_model(model_id, accelerator_count):
    """Uploads and deploys the model to Vertex AI endpoint for prediction."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_id}-endpoint")
    serving_env = {
        "MODEL_ID": model_id,
        "DEPLOY_SOURCE": "notebook",
    }

    model = aiplatform.Model.upload(
        display_name=model_id,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/transformers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )

    model.deploy(
        endpoint=endpoint,
        machine_type="n1-standard-8",
        accelerator_type="NVIDIA_TESLA_V100",
        accelerator_count=accelerator_count,
        deploy_request_timeout=3600,
    )
    return model, endpoint

## Upload and deploy models

This notebook and the serving can support all the 4 variations of the NLLB models listed below:


| Model Name | Model Type | #params | checkpoint |
| - | - | - | - |
| NLLB-200-3.3B | Dense | 3.3B |[model](https://huggingface.co/facebook/nllb-200-3.3B) |
| NLLB-200-1.3B | Dense | 1.3B |[model](https://huggingface.co/facebook/nllb-200-1.3B) |
| NLLB-200-Distilled-1.3B | Dense | 1.3B | [model](https://huggingface.co/facebook/nllb-200-distilled-1.3B) |
| NLLB-200-Distilled-600M | Dense | 600M | [model](https://huggingface.co/facebook/nllb-200-distilled-600M) |


Among the 4 variations, the [NLLB-200-3.3B](https://huggingface.co/facebook/nllb-200-3.3B) requires 2 V100 GPUs for online prediction. The rest of the variations, however, only require a single V100 GPU. This section demonstrate how to upload the pre-trained model to Model Registry and deploys it on the Endpoint with the corresponding machine specifications.

NOTE: The model deployment step will take ~25 minutes to complete.

In [ ]:
# Please select a model from the list:
model_id = "facebook/nllb-200-distilled-600M"  # @param ["facebook/nllb-200-distilled-600M", "facebook/nllb-200-distilled-1.3B", "facebook/nllb-200-1.3B", "facebook/nllb-200-3.3B"]

accelerator_count = None
# For models `NLLB-200-Distilled-600M`, `NLLB-200-Distilled-1.3B`, `NLLB-200-1.3B`,
# we use 1 V100 GPU
if model_id in [
    "facebook/nllb-200-distilled-600M",
    "facebook/nllb-200-distilled-1.3B",
    "facebook/nllb-200-1.3B",
]:
    accelerator_count = 1
# For model `NLLB-200-3.3B`, we require 2 V100 GPUs
elif model_id in ["facebook/nllb-200-3.3B"]:
    accelerator_count = 2


if model_id is None or accelerator_count is None:
    raise ValueError("Please select a valid model from the list.")

model, endpoint = deploy_model(model_id, accelerator_count)

NOTE: The model weights will be downloaded after the deployment succeeds. Thus additional 20 minutes of waiting time is needed **after** the above model deployment step succeeds and before you run the next step below. Otherwise you might see a `ServiceUnavailable: 503 502:Bad Gateway` error when you send requests to the endpoint.

## Send prediction request

In [ ]:
input_text = "Şeful ONU spune că nu există o soluţie militară în Siria"
# The source and the target languages. The full list of the languages (and the
# corresponding language codes) that are currently supported by the series of models,
# can be found at: https://github.com/facebookresearch/flores/blob/main/flores200/README.md#languages-in-flores-200
src_lang = "ron_Latn"
tgt_lang = "jpn_Jpan"

instances = [
    {"text": input_text, "src_lang": src_lang, "tgt_lang": tgt_lang},
]
preds = endpoint.predict(instances=instances).predictions
print(preds[0].get("translation_text"))

## Clean up

In [ ]:
# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete models.
model.delete()